In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install xlsxwriter
import os
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
from random import shuffle
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import to_categorical
import xlsxwriter

In [ ]:
import os
import cv2
import numpy as np
from sklearn.utils import shuffle
from tqdm import tqdm  # Install tqdm if not already installed: pip install tqdm
from keras.utils import to_categorical

# Set directories
Train_DIR = r"/content/drive/MyDrive/cpmbined data train"
TEST_DIR = r"/content/drive/MyDrive/final train and test/test"
result_excel_link = r"/content/drive/MyDrive/ResNet_epoch25.xlsx"
save_dir = r"/content/drive/MyDrive/DTS/models"
model_name = 'Alexnet_ecg_model_25epochs.keras'

# Constants
IMG_SIZE = 224
num_classes = 4
batch_size = 16
epochs = 25

test_img_num = 50  # 100 per class
classes = ['Normal Heartbeat', 'Abnormal Heartbeat', 'Myocardial Infarction (MI)', 'History of MI']
class_map = {label: i for i, label in enumerate(classes)}

In [ ]:


# Function to load data from folder structure with loading feedback
def load_data_from_directory(base_dir, classes, class_map, IMG_SIZE):
    data = []
    total_images = sum(len(os.listdir(os.path.join(base_dir, label))) for label in classes)

    print(f"Loading {total_images} images from '{base_dir}'...")

    # Use tqdm to create a progress bar
    with tqdm(total=total_images, unit="image") as pbar:
        for label in classes:
            path = os.path.join(base_dir, label)
            for img_name in os.listdir(path):
                try:
                    img_path = os.path.join(path, img_name)
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

                    # Check if image is successfully loaded
                    if img is None:
                        raise ValueError(f"Image {img_name} could not be read.")

                    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)  # Convert grayscale to RGB

                    # Append image, label, and name to data
                    data.append([np.array(img), class_map[label], img_name])

                    # Update progress bar
                    pbar.update(1)

                except Exception as e:
                    print(f"Error reading {img_name}: {e}")
                    pbar.update(1)  # Still update progress even if there's an error

    shuffle(data)
    print("Loading complete!")
    return data

# Load train and test data
train_data = load_data_from_directory(Train_DIR, classes, class_map, IMG_SIZE)
test_data = load_data_from_directory(TEST_DIR, classes, class_map, IMG_SIZE)

# Prepare training and testing datasets
trainImages = np.array([i[0] for i in train_data]).astype('float32') / 255.0  # Normalize pixel values
trainLabels = np.array([i[1] for i in train_data])  # Numeric labels
trainLabels = to_categorical(trainLabels, num_classes)  # Convert labels to one-hot encoding

testImages = np.array([i[0] for i in test_data]).astype('float32') / 255.0  # Normalize pixel values
testLabels = np.array([i[1] for i in test_data])  # Numeric labels
testLabels = to_categorical(testLabels, num_classes)  # Convert labels to one-hot encoding

print("Data preparation complete!")

In [ ]:
print(trainImages.shape, trainLabels.shape)
print(np.unique(np.argmax(trainLabels, axis=1)))  # Ensure label range is correct


In [ ]:
trainImages = trainImages.astype('float32') / 255.0
testImages = testImages.astype('float32') / 255.0


In [ ]:
# Learning rate scheduler
def lr_schedule(epoch, lr):
    if epoch > 15:
        return lr * 0.1
    elif epoch > 10:
        return lr * 0.5
    return lr


# Build AlexNet
model = Sequential()

# CONV Block 1
model.add(Conv2D(96, (11,11), strides=(4,4), input_shape=(224,224,3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(BatchNormalization())

# CONV Block 2
model.add(Conv2D(256, (11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(BatchNormalization())

# CONV Block 3
model.add(Conv2D(384, (3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
model.add(BatchNormalization())

# CONV Block 4
model.add(Conv2D(384, (3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
model.add(BatchNormalization())

# CONV Block 5
model.add(Conv2D(256, (3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(BatchNormalization())

# Fully Connected Block
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

# Output Layer
model.add(Dense(num_classes))
model.add(Activation('softmax'))


model.summary()

# Compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Callbacks
if not os.path.isdir(save_dir): os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', verbose=1, save_best_only=True)
lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
es = EarlyStopping(monitor='val_loss', patience=200, verbose=1)

callbacks = [checkpoint, lr_reducer, lr_scheduler, es]



from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    trainImages, trainLabels, test_size=0.2,
    stratify=np.argmax(trainLabels, axis=1),
    random_state=42
)

# Train
start_time = time.time()
history = model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val),
    shuffle=True,
    callbacks=callbacks
)

training_time = time.time() - start_time

# Load best model
best_model = load_model(filepath)

# Predict
preds = best_model.predict(testImages)

# Save results to Excel
workbook = xlsxwriter.Workbook(result_excel_link)
worksheet = workbook.add_worksheet()
worksheet.write(0, 0, "Filename")
worksheet.write(0, 1, "Original Label")
worksheet.write(0, 2, "Predicted Label")

for i, (img_arr, true_label, filename) in enumerate(test_data[:test_img_num]):
    pred_label = np.argmax(preds[i])
    worksheet.write(i + 1, 0, filename)
    worksheet.write(i + 1, 1, true_label)
    worksheet.write(i + 1, 2, pred_label)

workbook.close()

# Plot results
plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')

plt.subplot(2, 2, 2)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy')

plt.show()


In [ ]:
# Block 9: Evaluate the Model and Generate Metrics
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import load_model

# Load the best saved model
try:
    best_model = load_model(f'{model_type}_best.h5')
    print(f"Loaded best model from {model_type}_best.h5")
except Exception as e:
    print(f"Error loading best model: {e}. Using the current model instead.")
    best_model = model  # Fallback to the trained model if checkpoint loading fails

# Evaluate the model on test data
scores = best_model.evaluate(testImages, testLabels, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# Predict on test data
preds = best_model.predict(testImages)

# Convert predictions to class labels
pred_labels = np.argmax(preds, axis=1)

# Convert one-hot encoded testLabels to class indices
true_labels = np.argmax(testLabels, axis=1)

# Generate classification report
class_names = classes  # Use the defined classes ['N', 'L', 'R', 'A', 'V']
report = classification_report(true_labels, pred_labels, target_names=class_names)
print("Classification Report:\n")
print(report)

# Generate confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Save results to Excel
try:
    workbook = xlsxwriter.Workbook(result_excel_link)
    worksheet = workbook.add_worksheet()
    # Write headers
    worksheet.write(0, 0, "Filename")
    worksheet.write(0, 1, "Original Label")
    worksheet.write(0, 2, "Predicted Label")
    worksheet.write(0, 3, "Match (1/0)")
    worksheet.write(0, 4, "Total Execution Time")

    # Write data
    test_img_num = len(testImages)
    for i, (img_arr, true_label_numeric, filename) in enumerate(test_data[:test_img_num]):
        pred_label_numeric = np.argmax(preds[i])
        true_label = classes[true_label_numeric]
        pred_label = classes[pred_label_numeric]
        match = 1 if true_label == pred_label else 0
        worksheet.write(i +1, 0, filename)
        worksheet.write(i + 1, 1, true_label)
        worksheet.write(i + 1, 2, pred_label)
        worksheet.write(i + 1, 3, match)

    # Write total execution time
    end1 = time.time()
    time_taken = end1 - start1
    worksheet.write(1, 4, time_taken)

    # Close the workbook
    workbook.close()
    print(f"Results successfully written to {result_excel_link}")
except Exception as e:
    print(f"An error occurred while writing to Excel: {e}")

# Plot training results
plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')

plt.subplot(2, 2, 2)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy')

plt.show()